## Use this for 5 minute intervals matrix

In [ ]:
import pandas as pd
import requests
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import os
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to convert timestamp to date and time
def split_timestamp(timestamp):
    date, time_with_offset = timestamp.split('T')
    time = time_with_offset.split('+')[0]  # Removing the timezone offset
    return date, time

# Function to fetch rainfall data for a specific date
def fetch_rainfall_data(date):
    url = "https://api.data.gov.sg/v1/environment/rainfall"
    params = {'date': date}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json(), date
    else:
        print(f"Failed to fetch data for {date}, status code: {response.status_code}")
        return None, date

# Function to process data and save to Excel with conditional formatting
def process_and_save_data(result):
    data, date = result
    if data is None:
        print(f"No data to process for {date}")
        return

    stations = {station['id']: station for station in data['metadata']['stations']}
    rain_data = {}

    for item in data['items']:
        timestamp = item['timestamp']
        date, time = split_timestamp(timestamp)
        key = (date, time)
        
        if key not in rain_data:
            rain_data[key] = {}
            
        readings = item['readings']
        for reading in readings:
            station_id = reading['station_id']
            if station_id in stations:
                station_info = stations[station_id]
                rain_data[key][station_info['name']] = reading['value']
            else:
                print(f"Station ID {station_id} not found in metadata")

    # Create a DataFrame
    df = pd.DataFrame(rain_data).T  # Transpose to get dates and times as rows
    df.index = pd.MultiIndex.from_tuples(df.index, names=['Date', 'Time'])

    # Reorder columns to have station names in the correct order
    station_names = [stations[station_id]['name'] for station_id in stations]
    df = df.reindex(columns=station_names)

    # Reset index to have Date and Time as columns
    df = df.reset_index()

    # Create folder if it doesn't exist
    folder_name = 'API_2023' #<-----------------change the folder name here
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Save to Excel
    excel_filename = os.path.join(folder_name, f'{date}.xlsx')
    df.to_excel(excel_filename, index=False)

    # Load the workbook and the sheet
    wb = load_workbook(excel_filename)
    ws = wb.active

    # Define the fill for non-zero values
    highlight_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    # Apply conditional formatting
    for row in ws.iter_rows(min_row=2, min_col=3, max_row=ws.max_row, max_col=ws.max_column):
        for cell in row:
            if cell.value != 0 and cell.value is not None:
                cell.fill = highlight_fill

    # Save the workbook
    wb.save(excel_filename)
    print(f"Data for {date} has been saved to '{excel_filename}'")

# Main script to iterate through dates and process data
start_date = datetime.strptime('2023-01-01', '%Y-%m-%d') #<-----------------change the date here
end_date = datetime.strptime('2023-12-31', '%Y-%m-%d')  #<-----------------change the date here

dates = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# Using ThreadPoolExecutor to parallelize the fetching and processing of data
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_date = {executor.submit(fetch_rainfall_data, date.strftime('%Y-%m-%d')): date for date in dates}
    
    for future in as_completed(future_to_date):
        try:
            result = future.result()
            process_and_save_data(result)
        except Exception as e:
            date = future_to_date[future]
            print(f"Error processing data for {date.strftime('%Y-%m-%d')}: {e}")


## Run this to combine all the files from API into 1


Concate without summing

In [ ]:
import os
import pandas as pd

def read_and_format(file_path):
    """Read an Excel file and return a dictionary of DataFrames with combined 'Date' and 'Time' index or 'DateTime' index."""
    xls = pd.ExcelFile(file_path)
    sheet_dfs = {}
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        if 'Date' in df.columns and 'Time' in df.columns:
            df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))
            df = df.drop(columns=['Date', 'Time'])
        elif 'DateTime' in df.columns:
            df['DateTime'] = pd.to_datetime(df['DateTime'])
        else:
            print(f"Sheet {sheet_name} in file {file_path} does not have the expected date columns.")
            continue
        df = df.set_index('DateTime')
        sheet_dfs[sheet_name] = df
    return sheet_dfs

def process_files(folder_path):
    """Process each file in the folder and concatenate the data from all sheets."""
    combined_data = {}

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx'):  # Adjust the extension if your files are Excel files
            file_path = os.path.join(folder_path, file_name)
            try:
                sheet_dfs = read_and_format(file_path)
                for sheet_name, df in sheet_dfs.items():
                    if sheet_name not in combined_data:
                        combined_data[sheet_name] = []
                    combined_data[sheet_name].append(df)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

    # Combine data from each sheet separately
    for sheet_name in combined_data:
        combined_data[sheet_name] = pd.concat(combined_data[sheet_name])
        combined_data[sheet_name] = combined_data[sheet_name].sort_index()  # Sort by datetime index

    return combined_data

# Define the folder path
folder_path = r'C:\Users\userAdmin\Desktop\Codes\Data Extraction\API_2023'  # Update with the path to your folder

# Process the files and get the result
combined_data = process_files(folder_path)

# Save the result to a new Excel file with multiple sheets
output_file_path = '2023_spring_annual.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, df in combined_data.items():
        df.to_excel(writer, sheet_name=sheet_name)

print(f"Processed all files and saved the result to {output_file_path}")


Relative humidity


In [ ]:
import pandas as pd
import requests
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to convert timestamp to date and time
def split_timestamp(timestamp):
    date, time_with_offset = timestamp.split('T')
    time = time_with_offset.split('+')[0]  # Removing the timezone offset
    return date, time

# Function to fetch relative humidity data for a specific date
def fetch_relative_humidity_data(date):
    url = "https://api.data.gov.sg/v1/environment/relative-humidity"
    params = {'date': date}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json(), date
    else:
        print(f"Failed to fetch data for {date}, status code: {response.status_code}")
        return None, date

# Function to process data and save to Excel with interpolation for missing data
def process_and_save_data(result):
    data, date = result
    if data is None:
        print(f"No data to process for {date}")
        return

    stations = {station['id']: station for station in data['metadata']['stations']}
    humidity_data = {}

    for item in data['items']:
        timestamp = item['timestamp']
        date, time = split_timestamp(timestamp)
        key = (date, time)
        
        if key not in humidity_data:
            humidity_data[key] = {}
            
        readings = item['readings']
        for reading in readings:
            station_id = reading['station_id']
            if station_id in stations:
                station_info = stations[station_id]
                humidity_data[key][station_info['name']] = reading['value']
            else:
                print(f"Station ID {station_id} not found in metadata")

    # Create a DataFrame
    df = pd.DataFrame(humidity_data).T  # Transpose to get dates and times as rows
    df.index = pd.MultiIndex.from_tuples(df.index, names=['Date', 'Time'])

    # Reorder columns to have station names in the correct order
    station_names = [stations[station_id]['name'] for station_id in stations]
    df = df.reindex(columns=station_names)

    # Reset index to have Date and Time as columns
    df = df.reset_index()

    # Convert object columns to numeric
    df[station_names] = df[station_names].apply(pd.to_numeric, errors='coerce')

    # Interpolate missing data within each column and round to 1 decimal place
    df.interpolate(method='linear', axis=0, inplace=True)
    df = df.round(1)

    # Create folder if it doesn't exist
    folder_name = 'API_2023_Spring_Relative_Humidity' #<-----------------change the folder name here
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Create a new workbook
    excel_filename = os.path.join(folder_name, f'{date}_relative_humidity.xlsx')
    wb = Workbook()

    # Add original data to the first sheet
    ws1 = wb.active
    ws1.title = "Original Data"
    for r in dataframe_to_rows(df, index=False, header=True):
        ws1.append(r)

    # Group data into 5-minute intervals and add to a new sheet
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df.set_index('DateTime', inplace=True)

    # Create an empty DataFrame to store 5-minute grouped data
    df_5min = pd.DataFrame()

    for station in station_names:
        station_data = df[[station]].resample('5T').mean()
        df_5min = pd.concat([df_5min, station_data], axis=1)

    df_5min.reset_index(inplace=True)
    df_5min = df_5min.round(1)
    # Create a new sheet for 5-minute data
    ws2 = wb.create_sheet(title="5-Minute Data")
    for r in dataframe_to_rows(df_5min, index=False, header=True):
        ws2.append(r)

    # Save the workbook
    wb.save(excel_filename)
    print(f"Data for {date} has been saved to '{excel_filename}'")

# Main script to iterate through dates and process data
start_date = datetime.strptime('2023-01-01', '%Y-%m-%d') #<-----------------change the date here
end_date = datetime.strptime('2023-03-31', '%Y-%m-%d')  #<-----------------change the date here

dates = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# Using ThreadPoolExecutor to parallelize the fetching and processing of data
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_date = {executor.submit(fetch_relative_humidity_data, date.strftime('%Y-%m-%d')): date for date in dates}
    
    for future in as_completed(future_to_date):
        try:
            result = future.result()
            process_and_save_data(result)
        except Exception as e:
            date = future_to_date[future]
            print(f"Error processing data for {date.strftime('%Y-%m-%d')}: {e}")


Temperature

In [ ]:
import pandas as pd
import requests
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to convert timestamp to date and time
def split_timestamp(timestamp):
    date, time_with_offset = timestamp.split('T')
    time = time_with_offset.split('+')[0]  # Removing the timezone offset
    return date, time

# Function to fetch temperature data for a specific date
def fetch_temperature_data(date):
    url = "https://api.data.gov.sg/v1/environment/air-temperature"
    params = {'date': date}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json(), date
    else:
        print(f"Failed to fetch data for {date}, status code: {response.status_code}")
        return None, date

# Function to process data and save to Excel with interpolation for missing data
def process_and_save_data(result):
    data, date = result
    if data is None:
        print(f"No data to process for {date}")
        return

    stations = {station['id']: station for station in data['metadata']['stations']}
    temperature_data = {}

    for item in data['items']:
        timestamp = item['timestamp']
        date, time = split_timestamp(timestamp)
        key = (date, time)
        
        if key not in temperature_data:
            temperature_data[key] = {}
            
        readings = item['readings']
        for reading in readings:
            station_id = reading['station_id']
            if station_id in stations:
                station_info = stations[station_id]
                temperature_data[key][station_info['name']] = reading['value']
            else:
                print(f"Station ID {station_id} not found in metadata")

    # Create a DataFrame
    df = pd.DataFrame(temperature_data).T  # Transpose to get dates and times as rows
    df.index = pd.MultiIndex.from_tuples(df.index, names=['Date', 'Time'])

    # Reorder columns to have station names in the correct order
    station_names = [stations[station_id]['name'] for station_id in stations]
    df = df.reindex(columns=station_names)

    # Reset index to have Date and Time as columns
    df = df.reset_index()

    # Convert object columns to numeric
    df[station_names] = df[station_names].apply(pd.to_numeric, errors='coerce')

    # Interpolate missing data within each column and round to 1 decimal place
    df.interpolate(method='linear', axis=0, inplace=True)
    df = df.round(1)

    # Create folder if it doesn't exist
    folder_name = 'API_2023_Spring_Temperature' #<-----------------change the folder name here
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Create a new workbook
    excel_filename = os.path.join(folder_name, f'{date}_temperature.xlsx')
    wb = Workbook()

    # Add original data to the first sheet
    ws1 = wb.active
    ws1.title = "Original Data"
    for r in dataframe_to_rows(df, index=False, header=True):
        ws1.append(r)

    # Group data into 5-minute intervals and add to a new sheet
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df.set_index('DateTime', inplace=True)

    # Create an empty DataFrame to store 5-minute grouped data
    df_5min = pd.DataFrame()

    for station in station_names:
        station_data = df[[station]].resample('5T').mean()
        df_5min = pd.concat([df_5min, station_data], axis=1)

    df_5min.reset_index(inplace=True)
    df_5min = df_5min.round(1)  # Round 5-minute grouped data to 1 decimal place

    # Create a new sheet for 5-minute data
    ws2 = wb.create_sheet(title="5-Minute Data")
    for r in dataframe_to_rows(df_5min, index=False, header=True):
        ws2.append(r)

    # Save the workbook
    wb.save(excel_filename)
    print(f"Data for {date} has been saved to '{excel_filename}'")

# Main script to iterate through dates and process data
start_date = datetime.strptime('2023-01-01', '%Y-%m-%d') #<-----------------change the date here
end_date = datetime.strptime('2023-03-31', '%Y-%m-%d')  #<-----------------change the date here

dates = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# Using ThreadPoolExecutor to parallelize the fetching and processing of data
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_date = {executor.submit(fetch_temperature_data, date.strftime('%Y-%m-%d')): date for date in dates}
    
    for future in as_completed(future_to_date):
        try:
            result = future.result()
            process_and_save_data(result)
        except Exception as e:
            date = future_to_date[future]
            print(f"Error processing data for {date.strftime('%Y-%m-%d')}: {e}")
